# BERT and XLNet Fine-Tuned Notebook with UDA Options
## W266 Final Project
### Game of Thrones Text Classification
### T. P. Goter
### Fall 2019

This notebook is used to perform the baseline, finetuned BERT supervised text classification. The original UDA process utilized a Python script wrapped in a bash shell script. This notebook was generated in order to better show and annotate the process.

## Acknowledgement
Much of this code was leveraged from the open source [UDA](https://github.com/google-research/uda). It has been adapted to the Game of Thrones dataset. 

## Import Data Libraries

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import os
import collections
import tensorflow as tf
#tf.enable_eager_execution()
print(tf.__version__)

import yaml
import pprint

from absl import app
from absl import logging

# Mounting the drive is straightforward but required authentication each time 
# we reset the session
from google.colab import drive

drive.mount('/content/drive')
drive_path = '/content/drive/My Drive/NLP_FinalProject/working'


os.listdir(drive_path)



#
if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime!')
else:
  from google.colab import auth
  auth.authenticate_user()
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)
  
  # Upload credentials to TPU.
  with tf.Session(tpu_address) as sess:    
    with open('/content/adc.json', 'r') as f:
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(sess, credentials=auth_info)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)
  
  bucket_address = "gs://nlp_finalproject"


  assert bucket_address, 'Must specify an existing GCS bucket name'
  print('Using bucket: {}'.format(bucket_address))
  







1.15.0
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ERROR: Not connected to a TPU runtime!


## Get BERT onto Google Drive - DO THIS ONCE


In [0]:
#! mkdir /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained
# download bert base
#! wget -P /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
#! unzip /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained/uncased_L-12_H-768_A-12.zip &&
#! rm /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained/uncased_L-12_H-768_A-12.zip
#! mkdir /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained/bert_base
#! mv uncased_L-12_H-768_A-12 /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained/bert_base
#! mv /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained/bert_base/uncased*/* /content/drive/My\ Drive/NLP_FinalProject/working/bert_pretrained/bert_base/
#os.listdir(drive_path + '/bert_pretrained/bert_base/')

## Update Repository


In [3]:
!git -C /content/drive/My\ Drive/NLP_FinalProject/working pull

Already up to date.


## Create symbolic links to the Google Drive directories

In [4]:
!ln -s '/content/drive/My Drive/NLP_FinalProject/working/bert' bpath 
!ln -s '/content/drive/My Drive/NLP_FinalProject/working/' wpath 
!ln -s '/content/drive/My Drive/NLP_FinalProject/working/utils' upath 
!ln -s '/content/drive/My Drive/NLP_FinalProject/working/xlnet' xpath 

ln: failed to create symbolic link 'bpath/bert': Function not implemented
ln: failed to create symbolic link 'wpath/working': Function not implemented
ln: failed to create symbolic link 'upath/utils': Function not implemented
ln: failed to create symbolic link 'xpath/xlnet': Function not implemented


In [5]:
from bpath import modeling
from wpath import uda_colab
from upath import proc_data_utils, raw_data_utils, model_utils
from xpath import xlnet_colab

## Define Some Options
This section replaces passing the input parameters as command line arguments. This section is very important. It controls the entire model. See the dictionary below.

### Task Options:
- **do_train:** Boolean of whether we are training
- **do_eval:** Boolean of whether we are just evaluating

### Training Options:
- **sup_train_data_dir:** Input directory for supervised data. This should be set to "./Data/proc_data/train_##" where the ## is one of the subsets of training data generated from the prepro_ALL.csh script.
- **eval_data_dir:**  The input data dir of the evaluation data. This should be the path to the development data with which we will do hyperparameter tuning. We can change this to the test data directory once we are ready for final evaluation. The dev data path is: "./Data/proc_data/dev"
- **unsup_data_dir:** The input data dir of the unsupervised data. Path for the unsupervised, augmented data. This should be equal to "./Data/proc_data/unsup"
- **bert_config_file:** Absolute path to the json file corresponding to the pre-trained BERT model. For us this is: "./bert_pretrained/bert_base/bert_config.json"
- **vocab_file:** The vocabulary file that the BERT model was trained on. This should be equal to "./bert_pretrained/bert_base/vocab.txt"
- **init_checkpoint:** Initial checkpoint from the pre-trained BERT model. This should be equal to: "./bert_pretrained/bert_base/bert_model.ckpt"
- **task_name:** The name of the task to train. This should be equal to "GoT"
- **model_dir:** The output directory where the model checkpoints will be written. This will be set to "models" followed by a case specific identifier.

### Model configuration
- **use_one_hot_embeddings:** Boolean, default: True, If True, tf.one_hot will be used for embedding lookups, otherwise tf.nn.embedding_lookup will be used. On TPUs, this should be True since it is much faster."
- **max_seq_length":** Integer, default = 128, The maximum total sequence length after WordPiece tokenization. Sequences longer than this will be truncated, and sequences shorter than this will be padded. Note, GoT data was processed to be on-average close to this length to minimize lost data.
- **model_dropout:** Float, default = -1 (i.e., no dropout). Dropout rate for both the attention and the hidden states.

### Training hyper-parameters
- **train_batch_size:** Integer, default = 32. Based on the discussion [here](https://github.com/google-research/bert#out-of-memory-issues). 32 is probably the largest we can run with 11 GB of RAM while using BERT base with a maximum sequence length of 128.
- **eval_batch_size:** Integer, default = 8, "Base batch size for evaluation."
- **save_checkpoints_num:** Integer, default = 20, Number of checkpoints to save during training.
- **iterations_per_loop:** Integer, default = 200, Number of steps to make in each estimator call.
- **num_train_steps:** Integer, no default, number of training steps

### Optimizer hyperparameters
- **learning_rate:** Float, default = 2e-5, The initial learning rate for Adam Optimizer
- **num_warmup_steps:** Integer, no default, Number of warmup steps
- **clip_norm:** Float, default= 1.0, Gradient clip hyperparameter.

### UDA Options:
- **unsup_ratio:** Integer - ratio between unsupervised batch size and supervised batch size. If zero - dont use
- **aug_ops:** String - what augmentation procedure do you want to run
- **aug_copy:** Integer - how many augmentations per example are to be generated
- **uda_coeff:** Float - default 1 - This is the coefficient on the UDA loss. Basically you can rely more or less on the UDA loss during the supervised training. The UDA paper generally kept this at 1
- **tsa:** String - Annealing schedule to use. Options provided are "" none, linear_schedule, log_schedule, exp_schedule
- **uda_softmax_temp:** Float, default -1, A smaller temperature will accentuate differences in probabilities. Low temps were used in the UDA paper for cases with low numbers of labeled data, after masking out uncertain predictions.
- **uda_confidence_thresh:** Float, default -1, Threshold value above which the consistency loss term from the UDA is used. Basically ensures we are using loss from random guesses.

### TPU and GPU Options:
- **use_tpu:** Boolean - self-explanatory - it affects how the model is run. If we run in colab this could be important. False means use CPU or GPU. We will default to FALSE.
- **tpu_name:** String - address of the tpu
- **gcp_project:** String - project name when using TPU
- **tpu_zone:** String - can be set or detected
- **master:** Address of the TPU master, if applicable



### Defaults - Use Bert-based or XLNet-based

The defaults below should not be changed. Note that a config file will be read in after this in order to update these if desired.

In [0]:
lm = 'XLNET' # must be BERT or XLNET

if lm == 'BERT':
    defaults_file = drive_path + '/config/defaults.yml'
    data_dir = drive_path + '/Data/proc_data/GoT'
    warmup = 300
    steps = 3000
    lrate = 3e-5
    config_file= os.path.join(drive_path, 'bert_pretrained', 'bert_base','bert_config.json')
    init_checkpoint =  os.path.join(drive_path, 'bert_pretrained','bert_base','bert_model.ckpt')
    spiece_model_file = None
    vocab_file = os.path.join(drive_path, 'bert_pretrained','bert_base','vocab.txt')
elif lm == 'XLNET':
    defaults_file = os.path.join(drive_path, 'config','defaults-xlnet.yml')
    data_dir = os.path.join(drive_path, 'Data', 'proc_data','GoT_xlnet')
    warmup = 300
    steps = 3000
    lrate = 2e-5
    config_file= os.path.join(drive_path, 'xlnet_pretrained', 'xlnet_base','xlnet_config.json')
    init_checkpoint =  os.path.join(drive_path, 'xlnet_pretrained','xlnet_base','xlnet_model.ckpt')
    spiece_model_file= os.path.join(drive_path, 'xlnet_pretrained','xlnet_base', 'spiece.model')
    vocab_file = None
else:
    raise ValueError("lm must be equal to BERT or XLNET")
      
with open(defaults_file, 'r') as def_config:
    options = yaml.safe_load(def_config)
    


## Set the Case to Run
This will ensure that different configurations are being controlled and saved separately. Just load in the correct yaml file that specifies all of the parameters.

### Updated to include UDA parameters

In [0]:
train_set = 'train_5000'
batch_size = 12
hdo = 0.10
uda = True
tfidf = 3
ur = 1
udac = 1
udact = -1
cda_smt = -1

case_specifics = {'do_train': True,
'do_eval' : True,
'sup_train_data_dir': os.path.join(data_dir, train_set),
'eval_data_dir':  os.path.join(data_dir, 'dev'),
'model_dir':  os.path.join('./model', lm, train_set, "bs"+str(batch_size)+"_hdo{:2.0f}".format((hdo*100))),
'model_config_file': config_file,
'init_checkpoint': init_checkpoint,
'spiece_model_file': spiece_model_file,
'vocab_file': vocab_file,
'num_train_steps':  steps, 
'train_batch_size': batch_size,
'learning_rate':  lrate,
'num_warmup_steps':  warmup, 
'hidden_dropout': hdo,
'attention_dropout': 0.1,
'save_checkpoints_num' :10,
'use_tpu' :False,
'use_one_hot_embeddings': False,
'language_model': lm}

uda_options = {   'unsup_data_dir': os.path.join(data_dir, 'unsup'), 
                  'unsup_ratio': 1,
                  'tsa' : 'log_schedule',
                  'aug_ops': 'tf_idf-0.{}'.format(tfidf),
                  'aug_copy': 1,
                  'uda_softmax_temp': -1,
                  'uda_confidence_thresh':udact,
                  'uda_coeff': udac,
                  'model_dir': os.path.join('./model', lm, train_set, "uda_bs{}_hdo{:2.0f}_log_tf{}_ur{}".format(batch_size, (hdo*100), tfidf, ur))}

In [8]:
# merge dictionaries    
options.update(case_specifics)
if uda:
  options.update(uda_options)

#
print()
print("="*50 + "\nFull Listing of Options: \n" + "="*50)
pprint.pprint(options)


Full Listing of Options: 
{'adam_epsilon': 1e-08,
 'attention_dropatt': 0.1,
 'attention_dropout': 0.1,
 'aug_copy': 1,
 'aug_ops': 'tf_idf-0.3',
 'clamp_len': -1,
 'clip_norm': 1.0,
 'cls_scope': False,
 'data_dir': './Data',
 'decay_method': 'poly',
 'do_eval': True,
 'do_predict': False,
 'do_train': True,
 'eval_all_ckpt': True,
 'eval_batch_size': 8,
 'eval_data_dir': '/content/drive/My '
                  'Drive/NLP_FinalProject/working/Data/proc_data/GoT_xlnet/dev',
 'eval_split': 'dev',
 'gcp_project': False,
 'hidden_dropout': 0.1,
 'init': 'normal',
 'init_checkpoint': '/content/drive/My '
                    'Drive/NLP_FinalProject/working/xlnet_pretrained/xlnet_base/xlnet_model.ckpt',
 'init_range': 0.1,
 'init_std': 0.02,
 'is_regression': False,
 'iterations_per_loop': 1000,
 'language_model': 'XLNET',
 'learning_rate': 2e-05,
 'lr_layer_decay_rate': 1.0,
 'master': False,
 'max_save': 5,
 'max_seq_length': 128,
 'min_lr_ratio': 0.0,
 'model_config_file': '/content/drive

## Setup the Job
This section of the code grabs the right data and reads in the BERT config file. We also dump our configuration options to a JSON file in the model directory.

In [9]:
# Record informational logs
logging.set_verbosity(logging.INFO)

# GoT Processor Class has methods for retrieving text/train/dev datasets
processor = raw_data_utils.GoTProcessor()

# Read in the labels
label_list = processor.get_labels()

# Create the directory for the current model
tf.io.gfile.makedirs(options['model_dir'])

if options['language_model'] == 'BERT':
    # Read the BertConfig File
    config = modeling.BertConfig.from_json_file(
          options['model_config_file'],
          options['hidden_dropout'],
          options['attention_dropout'])
    tf.io.write_file(os.path.join(options['model_dir'], "OPTIONS.json"), json.dumps(options))
elif options['language_model'] == 'XLNET':
    # construct xlnet config and save to model_dir
    config = xlnet_colab.XLNetConfig(json_path=options['model_config_file'])
    config.to_json(os.path.join(options['model_dir'], "config.json"))

## Model Specific Setup

In [10]:
logging.info("warmup steps {}/{}".format(
      options['num_warmup_steps'], options['num_train_steps']))

# Specify where the checkpoints will be saved. This is just integer division between the total number of training steps and the number of checkpoints
options['save_checkpoints_steps'] = options['num_train_steps'] // options['save_checkpoints_num']

# Log the checkpoints
logging.info("setting save checkpoints steps to {:d}".format(
      options['save_checkpoints_steps']))

# Update iterations per loop
options['iterations_per_loop'] = min(options['save_checkpoints_steps'],
                                  options['iterations_per_loop'])

INFO:absl:warmup steps 300/3000
INFO:absl:setting save checkpoints steps to 300


## Setup Run Configuration

In [11]:
# Replaced old logic with xlnet logic - should work for BERT and XLNET now
run_config = model_utils.configure_tpu(options)


INFO:tensorflow:Single device mode.


INFO:tensorflow:Single device mode.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## Create our model
Our model uses the bert configuration parameters as well as gets fed all of our particular running options.

In [0]:
model_fn = uda_colab.model_fn_builder(
      options,
      config=config,
      num_labels=len(label_list),
      print_feature=False,
      print_structure=False,
      freeze_layers = (False, )
  )

## Create our Estimator
Basically contains the intstructions to run the model function.

In [13]:

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=options['use_tpu'],
    model_fn=model_fn,
    config=run_config,
    params={"model_dir": options['model_dir']},
    train_batch_size=options['train_batch_size'],
    eval_batch_size=options['eval_batch_size'],
    predict_batch_size=options['eval_batch_size'])



INFO:tensorflow:Using config: {'_model_dir': './model/XLNET/train_5000/uda_bs12_hdo10_log_tf3_ur1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f98d6d3e208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=300, num_shards=None, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_n

INFO:tensorflow:Using config: {'_model_dir': './model/XLNET/train_5000/uda_bs12_hdo10_log_tf3_ur1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f98d6d3e208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=300, num_shards=None, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_n

INFO:tensorflow:_TPUContext: eval_on_tpu True


INFO:tensorflow:_TPUContext: eval_on_tpu True


## Ready to Train

In [14]:
# Logical check to determine if we are training (vice evaluating)
if options['do_train']:
    logging.info("Getting data for supervised training from: {}".format(options['sup_train_data_dir']))
    
    # Are we doing UDA or just simple finetuning?
    if options['unsup_ratio'] > 0:
        logging.info("Getting unsupervised data from: {}".format(
          options['unsup_data_dir']))
    
    # Pass on all of the training sup/unsup options
    train_input_fn = proc_data_utils.training_input_fn_builder(
        options['language_model'],
        options['sup_train_data_dir'],
        max_seq_len=options['max_seq_length'],
        unsup_data_base_path=options['unsup_data_dir'],
        aug_ops=options['aug_ops'],
        aug_copy=options['aug_copy'],
        unsup_ratio=options['unsup_ratio'])

INFO:absl:Getting data for supervised training from: /content/drive/My Drive/NLP_FinalProject/working/Data/proc_data/GoT_xlnet/train_5000
INFO:absl:Getting unsupervised data from: /content/drive/My Drive/NLP_FinalProject/working/Data/proc_data/GoT_xlnet/unsup


INFO:tensorflow:loading training data from these files: /content/drive/My Drive/NLP_FinalProject/working/Data/proc_data/GoT_xlnet/train_5000/tf_examples.tfrecord.0.0 /content/drive/My Drive/NLP_FinalProject/working/Data/proc_data/GoT_xlnet/train_5000/tf_examples.tfrecord.0.1


INFO:tensorflow:loading training data from these files: /content/drive/My Drive/NLP_FinalProject/working/Data/proc_data/GoT_xlnet/train_5000/tf_examples.tfrecord.0.0 /content/drive/My Drive/NLP_FinalProject/working/Data/proc_data/GoT_xlnet/train_5000/tf_examples.tfrecord.0.1


INFO:tensorflow:Getting Augmented Data Files


INFO:tensorflow:Getting Augmented Data Files


...

INFO:tensorflow:exist_copy_num: 


INFO:tensorflow:Current number of files to process: 4


INFO:tensorflow:Current number of files to process: 4


INFO:tensorflow:Number of data files to process: 4


INFO:tensorflow:Number of data files to process: 4


In [15]:
# Logical check to see if we are evaluating against the development set (or test set if you change the eval_data_dir)
if options['do_eval']:
    logging.info("  >>> dev data dir : {}".format(options['eval_data_dir']))
    eval_input_fn = proc_data_utils.evaluation_input_fn_builder(
        options['language_model'],
        options['eval_data_dir'],
        "clas",
        max_seq_len=options['max_seq_length'])

    eval_size = processor.get_dev_size()
    eval_steps = int(eval_size / options['eval_batch_size'])
    logging.info("Evaluating {} examples in {} steps".format(eval_size,eval_steps))

INFO:absl:  >>> dev data dir : /content/drive/My Drive/NLP_FinalProject/working/Data/proc_data/GoT_xlnet/dev


INFO:tensorflow:loading eval clas data from these files: /content/drive/My Drive/NLP_FinalProject/working/Data/proc_data/GoT_xlnet/dev/tf_examples.tfrecord.0.0


INFO:tensorflow:loading eval clas data from these files: /content/drive/My Drive/NLP_FinalProject/working/Data/proc_data/GoT_xlnet/dev/tf_examples.tfrecord.0.0
INFO:absl:Evaluating 2500 examples in 312 steps


In [0]:
# IF we are training and evaluating
eval_results = []
if options['do_train'] and options['do_eval']:
    logging.info("***** Running training & evaluation *****")
    logging.info("  Supervised batch size = {:d}".format(
        options['train_batch_size']))
    logging.info("  Unsupervised batch size = {:d}".format(
        options['train_batch_size'] * options['unsup_ratio']))
    logging.info("  Num steps = {}".format(options['num_train_steps']))
    logging.info("  Base evaluation batch size = {:d}".format(
        options['eval_batch_size']))
    logging.info("  Num steps = {:d}".format(eval_steps))
    
    # Initialize
    best_acc = 0
    
    # Looping over training steps by subset (for each checkpoint)
    for s in range(0, options['num_train_steps'], options['save_checkpoints_steps']):
        logging.info("*** Running training: Current Step: {}***".format(s))
        
        estimator.train(
              input_fn=train_input_fn,
              steps=options['save_checkpoints_steps'])
        
        logging.info("*** Running evaluation ***")
        dev_result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
        logging.info(">> Results:")
        
        # Keep track of the evaluation results
        for key in dev_result.keys():
            logging.info("  {} = {}".format(key, str(dev_result[key])))
            dev_result[key] = dev_result[key].item()
        eval_results.append(dev_result)
        # Update the best accuracy object
        best_acc = max(best_acc, dev_result["eval_classify_accuracy"])
    logging.info("***** Final evaluation result *****")
    logging.info("Best acc: {:.3f}\n\n".format(best_acc))
    
    eval_results.sort(key=lambda x: x['eval_classify_accuracy'], reverse=True)

    tf.logging.info("=" * 80)
    log_str = "Best result | "
    for key, val in sorted(eval_results[0].items(), key=lambda x: x[0]):
      log_str += "{} {} | ".format(key, val)
    tf.logging.info(log_str)
    
elif options['do_train']:
    logging.info("***** Running training *****")
    logging.info("  Supervised batch size = {}".format(options['train_batch_size']))
    logging.info("  Unsupervised batch size = {}".format(
                    options['train_batch_size'] * options['unsup_ratio']))
    logging.info("  Num steps = {}".format(options['num_train_steps']))
    estimator.train(input_fn=train_input_fn, max_steps=options['num_train_steps'])
elif options['do_eval']:
    logging.info("***** Running evaluation *****")
    logging.info("  Base evaluation batch size = {}".format(options['eval_batch_size']))
    logging.info("  Num steps = {}".format(eval_steps))
    
    # Load in the checkpoint from training to do the evaluation
    checkpoint_state = tf.train.get_checkpoint_state(options['model_dir'])

    best_acc = 0
    for ckpt_path in checkpoint_state.all_model_checkpoint_paths:
        if not tf.io.gfile.exists(ckpt_path + ".data-00000-of-00001"):
            logging.info(
                "Warning: checkpoint {:s} does not exist".format(ckpt_path))
        continue
        logging.info("Evaluating {:s}".format(ckpt_path))
        dev_result = estimator.evaluate(
          input_fn=eval_input_fn,
          steps=eval_steps,
          checkpoint_path=ckpt_path,)
        logging.info(">> Results:")
        
        # keep track of evaluation metrics
        for key in dev_result.keys():
            logging.info("  {:s} = {:s}".format(key, str(dev_result[key])))
            dev_result[key] = dev_result[key].item()
        
        # update our best accuracy variable
        best_acc = max(best_acc, dev_result["eval_classify_accuracy"])
    logging.info("***** Final evaluation result *****")
    logging.info("Best acc: {:.3f}\n\n".format(best_acc))
    
    

INFO:absl:***** Running training & evaluation *****
INFO:absl:  Supervised batch size = 12
INFO:absl:  Unsupervised batch size = 12
INFO:absl:  Num steps = 3000
INFO:absl:  Base evaluation batch size = 8
INFO:absl:  Num steps = 312
INFO:absl:*** Running training: Current Step: 0***


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


...

INFO:tensorflow:global_step/sec: 0.437241


INFO:tensorflow:global_step/sec: 0.437241


INFO:tensorflow:examples/sec: 5.24689


INFO:tensorflow:examples/sec: 5.24689


INFO:tensorflow:global_step/sec: 0.434429


In [18]:
eval_results

[{'eval_classify_accuracy': 0.6854968070983887,
  'eval_classify_loss': 0.7649005055427551,
  'eval_mpca': 0.6948264837265015,
  'eval_precision': 0.959857702255249,
  'eval_recall': 0.9116795659065247,
  'global_step': 900,
  'loss': 1.1036555767059326},
 {'eval_classify_accuracy': 0.667067289352417,
  'eval_classify_loss': 0.8101212978363037,
  'eval_mpca': 0.6826959848403931,
  'eval_precision': 0.9716728925704956,
  'eval_recall': 0.8774131536483765,
  'global_step': 1200,
  'loss': 1.0531425476074219},
 {'eval_classify_accuracy': 0.667067289352417,
  'eval_classify_loss': 0.8179272413253784,
  'eval_mpca': 0.6826959848403931,
  'eval_precision': 0.9716728925704956,
  'eval_recall': 0.8774131536483765,
  'global_step': 1500,
  'loss': 0.9564679265022278},
 {'eval_classify_accuracy': 0.667067289352417,
  'eval_classify_loss': 0.8225918412208557,
  'eval_mpca': 0.6826959848403931,
  'eval_precision': 0.9716728925704956,
  'eval_recall': 0.8774131536483765,
  'global_step': 1800,
  'l

In [27]:
!ls ./model/XLNET/train_5000/


uda_bs12_hdo10_log_tf1_ur1  uda_bs24_hdo10_log_tf1_ur0.5
uda_bs12_hdo10_log_tf3_ur1


In [22]:
!mv ./model/XLNET/train_5000/uda_bs12_hdo10_log_tf1_ur1/checkpoint /content/drive/My\ Drive/NLP_FinalProject/working/model/XLNET/train_5000/uda_bs12_hdo10_log_tf1_ur1_lr3_3000
!mv ./model/XLNET/train_5000/uda_bs12_hdo10_log_tf1_ur1/events* /content/drive/My\ Drive/NLP_FinalProject/working/model/XLNET/train_5000/uda_bs12_hdo10_log_tf1_ur1_lr3_3000
!mv ./model/XLNET/train_5000/uda_bs12_hdo10_log_tf1_ur1/graph* /content/drive/My\ Drive/NLP_FinalProject/working/model/XLNET/train_5000/uda_bs12_hdo10_log_tf1_ur1_lr3_3000
!mv ./model/XLNET/train_5000/uda_bs12_hdo10_log_tf1_ur1/model.ckpt-1200* /content/drive/My\ Drive/NLP_FinalProject/working/model/XLNET/train_5000/uda_bs12_hdo10_log_tf1_ur1_lr3_3000

mv: cannot stat './model/XLNET/train_5000/uda_bs12_hdo10_log_tf1_ur1/model.ckpt-1200*': No such file or directory


In [0]:
!rm -rf ./model/XLNET

In [21]:
!ls /content/drive/My\ Drive/NLP_FinalProject/working/model/XLNET/train_5000/

bs32_hdo10_lr3	bs32_hdo15  uda_bs12_hdo10_log_tf1_ur1_lr3_3000
